### --- Day 21: Monkey Math ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/21

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 21);
WriteLines(inputLines);

Loading puzzle file: Day21.txt
Total lines: 2877
Max line length: 17

zfbd: 5
wnqf: pnwq * htpn
tdrp: wrpw * vvwp
clfw: rnzp * zvhb
gncs: 10


In [4]:
string[] testInputLines = 
[
    "root: pppw + sjmn",
    "dbpl: 5",
    "cczh: sllz + lgvd",
    "zczc: 2",
    "ptdq: humn - dvpt",
    "dvpt: 3",
    "lfqf: 4",
    "humn: 5",
    "ljgn: 2",
    "sjmn: drzm * dbpl",
    "sllz: 4",
    "pppw: cczh / lfqf",
    "lgvd: ljgn * ptdq",
    "drzm: hmdt - zczc",
    "hmdt: 32",
];

Ok, this one looks like we can model it with expression trees. We have the constant expression, and an arithmetic expression which in turn takes a left expression and a right expression. Then we ask the root expression for its answer and it will "pull" the results all the way back to the constants.

In [5]:
interface MonkeyExp {}

record ConstExp(int value) : MonkeyExp {}
record ArithExp(string left, string op, string right) : MonkeyExp {}

In [6]:
Dictionary<string, MonkeyExp> ParseMonkeyDict(string[] inputLines)
{
    Dictionary<string, MonkeyExp> result = new();

    foreach (var line in inputLines)
    {
        var lineBits = line.Replace(":", "").Split(' ').ToArray();
        var (id, exp) = parseExp(lineBits);
        result[id] = exp;
    }

    (string id, MonkeyExp exp) parseExp(string[] bits) => bits switch 
    {
        [var id, var c] => (id, new ConstExp(int.Parse(c))),
        [var id, var left, var op, var right] => (id, new ArithExp(left, op, right)),
        _ => throw new Exception("Unexpected line")
    };

    return result;
}

In [7]:
long FindRoot(string[] inputLines)
{
    var monkeyDict = ParseMonkeyDict(inputLines);

    long Calculate(MonkeyExp exp) => exp switch 
    {
        ConstExp c => CalcConst(c),
        ArithExp a => CalcArith(a),
        _ => throw new Exception("Unexpected expression")
    };

    long CalcConst(ConstExp e) => e.value;

    long CalcArith(ArithExp e) => e.op switch
    {
        "+" => Calculate(monkeyDict[e.left]) + Calculate(monkeyDict[e.right]),
        "-" => Calculate(monkeyDict[e.left]) - Calculate(monkeyDict[e.right]),
        "/" => Calculate(monkeyDict[e.left]) / Calculate(monkeyDict[e.right]),
        _ => Calculate(monkeyDict[e.left]) * Calculate(monkeyDict[e.right]),
    };

    return Calculate(monkeyDict["root"]);
}

In [8]:
// This process continues until root yells a number: 152.

var testAnswer = FindRoot(testInputLines);
Console.WriteLine(testAnswer);

152


In [9]:
// However, your actual situation involves considerably more monkeys. What
// number will the monkey named root yell?

var part1Answer = FindRoot(inputLines);
Console.WriteLine(part1Answer);


75147370123646


In [10]:
// 75147370123646 is correct!
Ensure(75147370123646, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/21

So, I think for this part, in its simplest form we're solving an equation such as $x + 1234 = 301$, where $1234$ is an arbitrary constant on the other side of an arithmetic expression. In actuality, $x$ is actually much further up the tree, eg:

$(x / 10) + 1234 = 301$

$((x / 10) - 50) + 1234 = 301$

Given all other values are constant, I believe we can flatten the tree so it's just a linear path of operations from $x$ to $301$. Then we can apply the inverse of each operation until we solve for $x$.

Oh but wait! What if the `humn` output is used by multiple operators?! Eg, we could have:

$(x + 10) * (x - 20) = 301$

We could be dealing with quadratic equations, or worse! Technically still solvable, but not by simply unwinding the operations. If this were the case, I think we would need to try another approach, like binary searching the input space.

A quick look at the input text makes me think it's linear, but it'll be best to confirm this before we get too deep.

#### _Many moments later..._ ####

Welp, another case where we got a correct answer for the sample input but not for the puzzle input! My intuitition was correct insofar as the equation to solve is linear, so why couldn't I get the correct answer? I had to resort to an LLM for some hints until I had the forehead-slap moment: I had ingored that for some of the arithmetic operations, our variable could be on the left or right side! That is, solving $x / 1 = 2$ is different to solving $1 / x = 2$. Once I factored that in, the solution finally emerged.

Another interesting aspect of this puzzle is that at no point does it specify whether we're performing integer division or floating-point division. Normally these puzzles will state the type of division when it's relevant. I had assumed integer division, but thanks to my oversight with operator ordering I did chase a red herring of floating-point precision. I also wondered whether integer arithmetic was actually the crux of the problem, i.e, the solution to $x / 3 = 3$ has three solutions, $9$, $10$, and $11$! That would have been super nasty! But thankfully, once my operator problem was solved, I switched back to `long` and confirmed the puzzle is indeed solvable with straightforward integer arithmetic.

In [12]:
using ValHumn = (long value, bool isHumn);
using Op = (string op, long value, bool leftIsHumn);

long FindRoot2(string[] inputLines)
{
    var monkeyDict = ParseMonkeyDict(inputLines);
    var humanExp = monkeyDict["humn"];
    var root = (ArithExp)monkeyDict["root"];
    List<Op> opList = new();

    var leftValue = calculate(monkeyDict[root.left]);
    var rightValue = calculate(monkeyDict[root.right]);

    long targetValue = leftValue.isHumn ? rightValue.value : leftValue.value;

    opList.Reverse();
    foreach (var (op, value, leftIsHumn) in opList)
    {
        targetValue = reverse(targetValue, op, value, leftIsHumn);
    }
    return targetValue;

    ValHumn calculate(MonkeyExp exp) => exp switch
    {
        ConstExp c => calcConst(c),
        ArithExp a => calcArith(a),
        _ => throw new Exception("Unexpected expression")
    };

    ValHumn calcConst(ConstExp c)
    {
        return (c.value, Object.ReferenceEquals(c, humanExp));
    }

    ValHumn calcArith(ArithExp a)
    {
        var leftValue = calculate(monkeyDict[a.left]);
        var rightValue = calculate(monkeyDict[a.right]);

        if (leftValue.isHumn && rightValue.isHumn)
        {
            throw new Exception("Non-linear equation!");
        }

        if (leftValue.isHumn)
        {
            opList.Add((a.op, rightValue.value, true));
        }

        if (rightValue.isHumn)
        {
            opList.Add((a.op, leftValue.value, false));
        }
        
        var resultValue = forward(leftValue.value, a.op, rightValue.value);
        var isHumn = leftValue.isHumn || rightValue.isHumn;
        return (resultValue, isHumn);
    }
}

long forward(long left, string op, long right) 
{
    var result = op switch 
    {
        "+" => left + right,
        "-" => left - right,
        "/" => left / right,
        _ => left * right,
    };

    return result;
}

long reverse(long targetValue, string op, long value, bool leftIsHumn) 
{
    return (op, leftIsHumn) switch
    {
        // 1 + x = 2, or // x + 1 = 2
        ("+", _) => targetValue - value,

        // x - 1 == 2
        ("-", true) => targetValue + value,
        // 1 - x == 2
        ("-", false) => -targetValue + value,

        // 1 * x == 2 or x * 1 == 2
        ("*", _) => targetValue / value,

        // x / 1 == 2
        ("/", true) => targetValue * value,
        // 1 / x == 2
        ("/", false) => value / targetValue,

        _ => throw new Exception("Unexpected operation")
    };
}

In [13]:
// In the above example, the number you need to yell to pass root's equality
// test is 301. (This causes root to get the same number, 150, from both of its
// monkeys.)

var part2TestResult  = FindRoot2(testInputLines);
Console.WriteLine(part2TestResult);

301


In [14]:
// What number do you yell to pass root's equality test?

var part2Answer = FindRoot2(inputLines);
Console.WriteLine(part2Answer);

3423279932937


In [15]:
// 3423279932937 is correct!
Ensure(3423279932937, part2Answer);